In [1]:
from pyspark import SparkConf, SparkContext
import nltk 

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1589975122815_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# To create Spark context and set Application name
sc=SparkContext.getOrCreate(SparkConf().setAppName("VocabularyExploration")) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# File locations in S3 (Modify to your S3 location)
dev_matched_file = 's3://comp5349-spam9926/assignment/dev_matched.tsv'
test_matched_file = 's3://comp5349-spam9926/assignment/test_matched.tsv'
dev_mismatched_file = 's3://comp5349-spam9926/assignment/dev_mismatched.tsv'
test_mismatched_file = 's3://comp5349-spam9926/assignment/test_mismatched.tsv'
train_file = 's3://comp5349-spam9926/assignment/train.tsv'

# read all the text files into variables
dev_matched_data = sc.textFile(dev_matched_file)
dev_mismatched_data = sc.textFile(dev_mismatched_file)
test_matched_data = sc.textFile(test_matched_file)
test_mismatched_data = sc.textFile(test_mismatched_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# function to read line which is tab delimted and return tuple([words,words])
def parseDataSentenceToWords(line):
    fields = line.split("\t")
    lower_case_setence = (fields[8]+" "+fields[9]).lower()
    return (nltk.word_tokenize(lower_case_setence))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# function to parse all the data into ([words,words]) and filter the header
def getParsedDataValues(parsed_data):
    parsed_data_header = parsed_data.first();
    parsed_data_values = parsed_data.filter(lambda x:x != parsed_data_header).map(parseDataSentenceToWords)
    return parsed_data_values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# parse the data from differnt data sets into ([words,words])
dev_matched_values = getParsedDataValues(dev_matched_data)
test_matched_values = getParsedDataValues(test_matched_data)
dev_mismatched_values = getParsedDataValues(dev_mismatched_data)
test_mismatched_values = getParsedDataValues(test_mismatched_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#join the dev and test matched values ([[word,word],[word,word]])
matched_values = dev_matched_values.union(test_matched_values)

#join the dev and test mismatched values ([[word,word],[word,word]])
mismatched_values = dev_mismatched_values.union(test_mismatched_values)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# flatten the list of lists to single list and remove duplicates by distinct
#ex: ([word,word,word])
matched_words = matched_values.flatMap(lambda x : (x)).distinct()
mismatched_words = mismatched_values.flatMap(lambda x : (x)).distinct() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#using intersection to find the common words between the matched and mismatched words
common_words = matched_words.intersection(mismatched_words).cache()

#subtract the common words from matched words
words_unique_to_matched_sets = matched_words.subtract(common_words)

#subtract the common words from mismatched words
words_unique_to_mismatched_sets = mismatched_words.subtract(common_words)

# count all the values
common_words_count = common_words.count()
word_un_mat_count = words_unique_to_matched_sets.count()
word_un_mismat_count  = words_unique_to_mismatched_sets.count()

print("Number Of Common Words Between Matched and Mismatched Set : ",common_words_count)
print("Number Of Unique Words In Matched Set : ",word_un_mat_count)
print("Number Of Unique Words In MisMatched Set  ",word_un_mismat_count)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number Of Common Words Between Matched and Mismatched Set :  9485
Number Of Unique Words In Matched Set :  10595
Number Of Unique Words In MisMatched Set   8242

In [13]:
# read train data set and read the first line as header to filter the header and 
train_data = sc.textFile(train_file)
header = train_data.first();

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# parse data function to parse the line and read only the genre(field3) and sentence1&2 (field8&9) 
# and tokenize the combined sentence to return as tuple (genre,[words])
def parseDataSentenceToGenreWords(line):
    fields = line.split("\t")
    lower_case_setence = (fields[8]+" "+fields[9]).lower()
    return (fields[3],nltk.word_tokenize(lower_case_setence))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# filter the header and map values in (genre, list_of_tokenised_words) format 
genre_words_values = train_data.filter(lambda x:x != header).map(parseDataSentenceToGenreWords).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# map them into (genre,word),(genre,word)
genre_word_with_stopwords = genre_words_values.flatMapValues(lambda x:x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# remove stop words - used nltk stopwords and  map them into (genre,word),(genre,word)
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
genre_word_without_stopwords = genre_words_values.flatMapValues(lambda x : [w for w in x if not w in stop_words])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def processPercentagesOfWords(genre_words_values):
   
    # now group by value i.e word - (word , [genre1,genre2]),(word , [genre3,genre2]). 
    # it will help with the word appearing in how may genres - ResultIterable

    grouped_genre_by_word = genre_words_values.groupBy(lambda x: x[1])
    
    # now count the values and map them into [(word, how_many_genres_the_word_appeared_in)]
    # ex: [('work', 5), ('out', 5)]
    grouped_total_genres_by_word = grouped_genre_by_word.mapValues(lambda vals: len(set(vals))).cache()
    
    # now count the total words in all the genres 
    total_word_count = grouped_total_genres_by_word.count();
   
    # now swap (word,number_of_genres_the word_appeared_in) to (number_of_genres_the word_appeared_in,word) 
    grouped_word_by_total_genres = grouped_total_genres_by_word.map(lambda x : (x[1],x[0])) 
    
    # count by keys so you get the total number of words appeared in different genres
    total_words_in_each_genre = grouped_word_by_total_genres.countByKey()
    
    print(" total words in all genres = ",total_word_count)
    print(" Percentage of words in 5 genres - ", (total_words_in_each_genre[5]/total_word_count)*100)
    print(" Percentage of words in 4 genres - ", (total_words_in_each_genre[4]/total_word_count)*100)
    print(" Percentage of words in 3 genres - ", (total_words_in_each_genre[3]/total_word_count)*100)
    print(" Percentage of words in 2 genres - ", (total_words_in_each_genre[2]/total_word_count)*100)
    print(" Percentage of words in 1 genres - ", (total_words_in_each_genre[1]/total_word_count)*100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
processPercentagesOfWords(genre_word_with_stopwords)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 total words in all genres =  85621
 Percentage of words in 5 genres -  8.988449095432195
 Percentage of words in 4 genres -  6.198245757466042
 Percentage of words in 3 genres -  7.953656229196109
 Percentage of words in 2 genres -  13.52238352740566
 Percentage of words in 1 genres -  63.337265390499994

In [20]:
processPercentagesOfWords(genre_word_without_stopwords)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 total words in all genres =  85475
 Percentage of words in 5 genres -  8.850541093887102
 Percentage of words in 4 genres -  6.204153261187482
 Percentage of words in 3 genres -  7.964902018133957
 Percentage of words in 2 genres -  13.540801403919275
 Percentage of words in 1 genres -  63.43960222287218